<div id="toc"></div>

The following code is simply to generate a table of contents.

In [1]:
%%javascript 
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

# Introduction
This guide is for those that would like to create models in Python to create new metrics in Lily, according to the diagram below. Especially since generating PMML files from models in Python is a bit tricky, this overview might be of some use.

I will go over most steps, from exporting dataframes out of Lily, to creating a model to predict a new metric in Python, to importing that metric back into Lily using PMML. I make use of several tools and packages, such as Jupyter Notebook, Pandas, Sklearn and a PMML-generator, which I will try to describe in some detail.

For this demo, I created a model that predicts the monthly income of a customer based on socio demographic metrics. This simple regression model is only used for the sake of demonstration, so I did not put much effort into issues like handling missing data, feature engineering, etc. The goal is simply to provide a possible workflow for model metric creation, and to add Python to the diagram below.

![](loop.png)

# Export Dataframe out of Lily
Generating a dataframe via the User Interface is pretty straightforward and well-documented. Just click on the '+' sign in the Dataframe tab to create a new dataframe, and provide a name and description. The set selection lets you choose which customers are included – i.e. the rows. The metric selection will take up the columns. For this demo, I simply selected all Socio Demo metrics, including the monthly income we would like to predict. It is important to enable the header option in the advanced options section as that will make life much easier.

After hitting ‘Save’, a dataframe will be generated which might take several minutes. At the status tab, a download icon will appear eventually. The download will consist of four separate files, with ‘dataframe.csv’ being the one we are interested in.

# Setting up a Python Environment
You should have installed [Python](https://www.python.org/downloads/) (2 or 3, as you like), [Git](https://git-scm.com/download/mac) and [Pip](https://pip.pypa.io/en/stable/installing/). The latter is a tool for installing Python packages. Simply download the get-pip.py script and run ‘python get-pip.py’ in the commander where you downloaded the script. Now you can install any packages running 'pip install -package-'. You should have these packages installed:

* 'pip install jupyter notebook' –  a handy Python environment
* 'pip install scipy' – providing scientific computing tools
* 'pip install pandas' – providing dataframe functionality as in R
* 'pip install scikit-learn' – providing model functionality
* 'pip install sklearn_pandas' – providing necessary tools for generating PMML

Some of these might require dependencies that will be installed automatically, but that might not be the case. If so, please install the required packages as well. If you successfully installed the packages above, you should be good to go.

You should also install sklearn2pmml by running following command. It is a package that generates a PMML file from an Sklearn object.

* 'pip install --user git+https://github.com/jpmml/sklearn2pmml.git'

# Jupyter Notebook
Jupyter notebook is a very accessible Python environment, consisting of text and code cells. If you are familiar with R Markdown, this is Python’s equivalent. Simply run ‘jupyter notebook’ in the terminal (at the directory where your ‘dataframe.csv’ is located) to fire it up. From there, hit the ‘New’ drop menu at the right, and ‘Python 2 or 3’ to create a new notebook. A typical notebook contains code and text cells alternating each other, where the code cells are being explained by text cells written in Markdown. You can try the notebook in your browser [here](https://try.jupyter.org/).

If you are not familiar with Jupyter Notebook, it might be a bit daunting at first, but it really pays off to get to know the basics and the shortcuts in particular. There are many relevant tutorials out there, but basically, there are two modes: command mode and edit mode. Command mode lets you play around on the level of the cells, like selecting, adding, removing text and code cells. Edit mode lets you edit one cell in particular. This is a list of shortcuts that I for one use all the time. 

* Press Esc and Enter to switch between Command and Edit mode.
* Press H to see a list of all shortcuts.
* In command mode:
    * Press B or A to create a new cell below or above
    * Press Arrow up and down to navigate
    * Press D twice to delete a cell
    * Press Y or M to alternate between code or text cells
* In Edit mode:
    * Press Shift + Enter to run a cell, either text or code

The next part will contain the actual code to import the dataframe in Python, to create a model, and to generate a PMML file.

# The Code
First, we need to import the necessary packages. Pandas will give us dataframe functionality as in R. Matplotlib will be used to generate some plots.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

## Import Dataframe
When a dataframe is generated in Lily's User Interface, the name of the actual data file is always 'dataframe.csv' and the delimiter is by default set on a vertical bar. So to import the data into a Pandas dataframe, we can run the following code. Note that I have located the dataframe file in the same folder as this notebook. You should provide a path if that is not the case for you. 

Finally, the tail function just shows us the final rows for the sake of illustration. The head function could be used just as well, but I generally prefer the tail function as it lets you check if all rows are included and nothing is wrong with the data.

In [3]:
rawData = pd.read_csv("dataframe.csv", delimiter="|")
rawData.tail(10)

,ID,referenceTime,originalSourceIds,title,lastName,firstName,middleName,streetAddress,state,zipcode,...,totalAssets,age,ageCategory,crmId,flagElitesMember,flagSelfSufficientsMember,flagNewWorldlyAdoptersMember,flagSafetySeekersMember,flagTraditionalistsMember,flagUpwardlyMobilesMember
50008,-4619305595032841619,2017-01-26T13:53:55.288Z,"[""""CRM:20439""""]",Mr.,Gould,Ken,A,4756 Warner Street,FL,33137.0,...,96743.0,31.0,18-34,20439.0,0,0,1,0,0,0
50009,-4619282005108468590,2017-01-26T13:53:55.288Z,"[""""CRM:3843""""]",Mr.,Durand,David,E,363 Tator Patch Road,IL,60455.0,...,116960.0,33.0,18-34,3843.0,0,0,0,0,0,0
50010,-4618812785648630615,2017-01-26T13:53:55.288Z,"[""""CRM:27810""""]",Mr.,McCutcheon,Lloyd,C,3028 Locust Court,CA,90706.0,...,37051.0,29.0,18-34,27810.0,0,0,0,0,0,0
50011,-4618783767582262908,2017-01-26T13:53:55.288Z,"[""""CRM:24247""""]",Ms.,Barr,Gloria,D,3048 Pinewood Drive,IL,60606.0,...,232889.0,24.0,18-34,24247.0,0,0,0,0,0,0
50012,-4618210187121199517,2017-01-26T13:53:55.288Z,"[""""CRM:7639""""]",Mr.,Vo,Carlos,I,3232 Ashcraft Court,CA,92111.0,...,148206.0,20.0,18-34,7639.0,0,0,0,0,0,0
50013,-4618209560386695881,2017-01-26T13:53:55.288Z,"[""""CRM:16017""""]",Ms.,Lacefield,Mary,P,1776 Patterson Street,TX,77002.0,...,29987.0,20.0,18-34,16017.0,0,0,0,0,0,0
50014,-4618028639641339898,2017-01-26T13:53:55.288Z,"[""""CRM:26883""""]",Mr.,Walker,Warren,L,498 Willison Street,MN,55429.0,...,28607.0,31.0,18-34,26883.0,0,0,0,0,0,0
50015,-4617960797035284685,2017-01-26T13:53:55.288Z,"[""""CRM:43096""""]",Dr.,Czajkowski,Eric,C,2246 Bicetown Road,NY,10013.0,...,25481.0,86.0,50 or older,43096.0,0,0,0,0,0,0
50016,-4617721746446202455,2017-01-26T13:53:55.288Z,"[""""CRM:10273""""]",Mrs.,Harris,Jana,J,1548 Watson Street,NJ,8102.0,...,38964.0,85.0,50 or older,10273.0,0,0,0,0,1,0
50017,-4616424157466986398,2017-01-26T13:53:55.288Z,"[""""CRM:22798""""]",Ms.,Foye,Jennie,R,2184 Heliport Loop,IN,47631.0,...,30969.0,40.0,35-49,22798.0,0,0,0,0,1,0


## Feature Engineering
There are quite a few categorical variables in our data that are as such not useful for a regression model. It is possible though to change such a variable into multiple binary variables, one for every value the original variable can take. This is something called one hot encoding. Because some variables (like zipcode) can take up many, many values, we would like to limit this number (as we would end up with too many binary features otherwise).

With the following code, we loop through the columns and verify the number of values that variable can take. If it is within the range of 3 and 10 (and if it is an object and not already a string), we store that variable in a list called 'dummies'. These are the variables we would like to encode into binary features per value.

In [6]:
dummies, nDummies = [], []
for i in range(len(rawData.columns)):
    possibleValues = len(rawData.iloc[:,i].unique())
    feature = rawData.columns[i]
    featureType = rawData.iloc[:,i].dtype
    if featureType == 'object' and possibleValues in range(2, 10):
        dummies.append(feature)
        print(feature, rawData.iloc[:,i].unique())
    else:
        nDummies.append(False)

title ['Mr.' 'Mrs.' 'Ms.' 'Dr.' nan]
gender ['male' 'female' nan]
lifestage ['Mature Years' 'Younger Years Fiscal Fledlings' 'Younger Years'
 'Family Life' nan]
maritalStatus ['Married' 'Single (never in legal unions)' 'In registered partnership'
 'Divorced' 'Widowed' nan]
education ['associate' 'bachelor' 'doctoral' 'master' 'secondary' 'elementary'
 'first-professional degree' nan]
resCountry ['US' nan]
ageCategory ['50 or older' '35-49' '18-34' nan]


For now, we use the Pandas get_dummies() function to encode these dummy variables.

In [7]:
data = pd.get_dummies(rawData, columns=dummies)

The result is a binary feature for every category the original feature could take, as we can see here.

In [8]:
data.columns

Index(['ID', 'referenceTime', 'originalSourceIds', 'lastName', 'firstName',
       'middleName', 'streetAddress', 'state', 'zipcode', 'emailAddress',
       'phoneNumber', 'flagAlive', 'occupation', 'jobSector', 'resCity',
       'cityPopulation', 'cityDensity', 'mainNationality', 'primaryLanguage',
       'flagStudent', 'flagRetired', 'flagEmployed', 'socialClassInd',
       'cohort', 'monthlyIncomeReal', 'nbrChildren', 'flagHouseOwner',
       'flagHomeTenant', 'flagValueProperty', 'flagLandlord',
       'flagWealthyRegion', 'residenceType', 'nbrPropertiesOwned',
       'totalAssets', 'age', 'crmId', 'flagElitesMember',
       'flagSelfSufficientsMember', 'flagNewWorldlyAdoptersMember',
       'flagSafetySeekersMember', 'flagTraditionalistsMember',
       'flagUpwardlyMobilesMember', 'title_Dr.', 'title_Mr.', 'title_Mrs.',
       'title_Ms.', 'gender_female', 'gender_male', 'lifestage_Family Life',
       'lifestage_Mature Years', 'lifestage_Younger Years',
       'lifestage_Younger 

## Feature Selection
For this demo, we would like to create a regression model to predict the monthly income of a customer. Therefore, we can only use numerical features (continuous or binary). Note that we could map categorical values into numerical ones, but here we do not bother with that, we simply delete them.

The describe function is a handy way to filter on numerical values.

In [9]:
data.describe()

,ID,zipcode,flagAlive,jobSector,cityPopulation,cityDensity,mainNationality,primaryLanguage,flagStudent,flagRetired,...,education_bachelor,education_doctoral,education_elementary,education_first-professional degree,education_master,education_secondary,resCountry_US,ageCategory_18-34,ageCategory_35-49,ageCategory_50 or older
count,5.001800e+04,50000.000000,50018.0,0.0,0.0,0.0,0.0,0.0,4.073000e+03,1.293300e+04,...,50018.000000,50018.000000,50018.000000,50018.000000,50018.000000,50018.000000,50018.000000,50018.000000,50018.00000,50018.000000
mean,-4.426309e+15,50738.002460,1.0,NaN,NaN,NaN,NaN,NaN,-5.442624e+303,-5.433686e+303,...,0.292015,0.010936,0.051142,0.016634,0.280279,0.141449,0.999640,0.370846,0.27742,0.351374
std,5.320400e+18,29274.147692,0.0,NaN,NaN,NaN,NaN,NaN,inf,inf,...,0.454694,0.104003,0.220289,0.127897,0.449140,0.348488,0.018967,0.483036,0.44773,0.477404
min,-9.222570e+18,758.000000,1.0,NaN,NaN,NaN,NaN,NaN,-5.486124e+303,-5.486124e+303,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
25%,-4.604410e+18,27801.000000,1.0,NaN,NaN,NaN,NaN,NaN,-5.449181e+303,-5.443883e+303,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.00000,0.000000
50%,-3.102969e+16,48342.000000,1.0,NaN,NaN,NaN,NaN,NaN,-5.449181e+303,-5.443883e+303,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.00000,0.000000
75%,4.616544e+18,77002.000000,1.0,NaN,NaN,NaN,NaN,NaN,-5.449181e+303,-5.406339e+303,...,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.00000,1.000000
max,9.222953e+18,99925.000000,1.0,NaN,NaN,NaN,NaN,NaN,-5.406321e+303,-5.401023e+303,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000


In [10]:
variables = list(data.describe())
data = data[variables]

Although the variables 'ID', 'zipcode' and 'crmId' are numerical, they do not hold any real numerical value. Therefore, including them in a regression model would be a mistake, so we should drop these as well.

In [11]:
data.drop(['ID', 'zipcode', 'crmId'], axis=1, inplace=True)
data.tail()

,flagAlive,jobSector,cityPopulation,cityDensity,mainNationality,primaryLanguage,flagStudent,flagRetired,flagEmployed,socialClassInd,...,education_bachelor,education_doctoral,education_elementary,education_first-professional degree,education_master,education_secondary,resCountry_US,ageCategory_18-34,ageCategory_35-49,ageCategory_50 or older
50013,1,NaN,NaN,NaN,NaN,NaN,-5.449181e+303,NaN,NaN,NaN,...,1,0,0,0,0,0,1,1,0,0
50014,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,0,0,0,0,0,1,1,0,0
50015,1,NaN,NaN,NaN,NaN,NaN,NaN,-5.443883e+303,NaN,NaN,...,0,0,0,0,0,0,1,0,0,1
50016,1,NaN,NaN,NaN,NaN,NaN,NaN,-5.443883e+303,NaN,NaN,...,0,0,0,0,0,0,1,0,0,1
50017,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,1,1,0,1,0


## Remove Missing Values
Before creating any model, we should deal with missing values first. The following code counts the missing values per variable.

In [12]:
data.isnull().sum()

flagAlive                                           0
jobSector                                       50018
cityPopulation                                  50018
cityDensity                                     50018
mainNationality                                 50018
primaryLanguage                                 50018
flagStudent                                     45945
flagRetired                                     37085
flagEmployed                                    50018
socialClassInd                                  50018
cohort                                          50018
monthlyIncomeReal                                  18
nbrChildren                                        18
flagHouseOwner                                  21548
flagHomeTenant                                  28488
flagValueProperty                               41111
flagLandlord                                    47276
flagWealthyRegion                               42321
residenceType               

For this demo, I won't be bothering with imputing any data. To simply drop data, we can use Pandas .dropna() function. First, we simply delete the columns with more than 30000 missing values by setting the axis to 1 - for columns - and setting a threshold. Next, we delete the rows containing any missing values for other variables as well, this time setting the axis to 0. The 'inplace' attribute simply means that the dataframe is stored in the same variable.

In [13]:
data.dropna(axis=1, thresh=1000, inplace=True)

In [14]:
data.flagStudent = data.flagStudent.apply(lambda x: 0 if np.isnan(x) else 1)
data.flagRetired = data.flagRetired.apply(lambda x: 0 if np.isnan(x) else 1)
data.flagHouseOwner = data.flagHouseOwner.apply(lambda x: 0 if np.isnan(x) else 1)
data.flagHomeTenant = data.flagHomeTenant.apply(lambda x: 0 if np.isnan(x) else 1)
data.flagValueProperty = data.flagValueProperty.apply(lambda x: 0 if np.isnan(x) else 1)
data.flagLandlord = data.flagLandlord.apply(lambda x: 0 if np.isnan(x) else 1)
data.flagWealthyRegion = data.flagWealthyRegion.apply(lambda x: 0 if np.isnan(x) else 1)

The result is a clean slate.

In [15]:
data.isnull().sum()

flagAlive                                        0
flagStudent                                      0
flagRetired                                      0
monthlyIncomeReal                               18
nbrChildren                                     18
flagHouseOwner                                   0
flagHomeTenant                                   0
flagValueProperty                                0
flagLandlord                                     0
flagWealthyRegion                                0
nbrPropertiesOwned                              18
totalAssets                                     18
age                                             18
flagElitesMember                                 0
flagSelfSufficientsMember                        0
flagNewWorldlyAdoptersMember                     0
flagSafetySeekersMember                          0
flagTraditionalistsMember                        0
flagUpwardlyMobilesMember                        0
title_Dr.                      

## Impute Missing Values

In [16]:
from sklearn.preprocessing import Imputer
imp = Imputer()

data = pd.DataFrame(imp.fit_transform(data), columns=data.columns)
data.isnull().sum().sum()

0

## Training and Validation

Before splitting the data into a training and validation set, some formatting is needed. As for our training data, we need a dataframe with all training variables included, but the target variable excluded. The latter should be stored in a Pandas Series. 

In [15]:
targets = data.loc[:,'monthlyIncomeReal']
data.drop('monthlyIncomeReal', axis=1, inplace=True)

Now we can use sklearn's built-in train_test_split function. Basically, it shuffles the data, and slices it according to given ratio. Here, we reserve 70% for training and 30% for validation.

In [16]:
from sklearn.model_selection import train_test_split
trainX, validX, trainY, validY = train_test_split(data, targets, train_size=0.7, random_state=100)

In [17]:
trainX.tail()

,flagAlive,flagStudent,flagRetired,nbrChildren,flagHouseOwner,flagHomeTenant,flagValueProperty,flagLandlord,flagWealthyRegion,nbrPropertiesOwned,...,education_bachelor,education_doctoral,education_elementary,education_first-professional degree,education_master,education_secondary,resCountry_US,ageCategory_18-34,ageCategory_35-49,ageCategory_50 or older
16304,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
79,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
12119,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
14147,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,3.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
38408,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [18]:
trainY.tail()

16304    2295.40
79       2732.05
12119    1535.55
14147    3685.05
38408    2445.61
Name: monthlyIncomeReal, dtype: float64

## Cross Validation Demo
When cross-validating, one might be tempted to use all training data. However, it is better to set apart a test set.

In [19]:
from sklearn import linear_model
from sklearn import neighbors
from sklearn.model_selection import cross_val_score, KFold

cv = KFold(n_splits=10, shuffle=True, random_state=1)
print(np.mean(cross_val_score(linear_model.LinearRegression(), trainX, trainY, cv=cv)))
print(np.mean(cross_val_score(linear_model.HuberRegressor(), trainX, trainY, cv=cv)))
print(np.mean(cross_val_score(linear_model.Lasso(alpha=1), trainX, trainY, cv=cv)))
print(np.mean(cross_val_score(linear_model.Ridge(alpha=2), trainX, trainY, cv=cv)))
print(np.mean(cross_val_score(neighbors.KNeighborsRegressor(), trainX, trainY, cv=cv)))

0.957420516768
0.493050125026
0.957128624875
0.957420804838
0.744354224974


In [20]:
for n in [10, 15, 20, 25, 30]:
    print(np.mean(cross_val_score(neighbors.KNeighborsRegressor(n_neighbors=n), trainX, trainY, cv=cv)))

0.763233939035
0.769807521396
0.772359839191
0.773970343575
0.775495410835


## Model Building

Sklearn lets you create all kinds of model objects - such as .LinearRegression() which can be used for fitting, scoring, etc. Here, we create several model objects, all using different machine learning algorithms. The idea is to let them compete against eachother, and finally preserve the best scoring model. In order to generate a PMML file later on, it is necessary to feed the sklearn objects into a PMMLPipeline function, provided by the sklearn2pmml package.

In [21]:
from sklearn2pmml import PMMLPipeline
from sklearn import linear_model
from sklearn import neighbors

lm = PMMLPipeline([("", linear_model.LinearRegression())])
hr = PMMLPipeline([("", linear_model.HuberRegressor())])
lr = PMMLPipeline([("", linear_model.Lasso(alpha=1))])
rr = PMMLPipeline([("", linear_model.Ridge(alpha=1))])
nn = PMMLPipeline([("", neighbors.KNeighborsRegressor())]) 

Now we can simply apply the .fit() function on these objects, with the training data and targets as parameters. This is the magic stuff.

In [22]:
lm.fit(trainX, trainY)
hr.fit(trainX, trainY)
lr.fit(trainX, trainY)
rr.fit(trainX, trainY)
nn.fit(trainX, trainY)

PMMLPipeline(steps=[('', KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform'))])

Next, we can try to predict on the validation set to see how well we're doing. The .predict() function returns an array with all predictions:

In [27]:
rr.active_fields

array(['flagAlive', 'flagStudent', 'flagRetired', 'nbrChildren',
       'flagHouseOwner', 'flagHomeTenant', 'flagValueProperty',
       'flagLandlord', 'flagWealthyRegion', 'nbrPropertiesOwned',
       'totalAssets', 'age', 'flagElitesMember',
       'flagSelfSufficientsMember', 'flagNewWorldlyAdoptersMember',
       'flagSafetySeekersMember', 'flagTraditionalistsMember',
       'flagUpwardlyMobilesMember', 'title_Dr.', 'title_Mr.', 'title_Mrs.',
       'title_Ms.', 'gender_female', 'gender_male',
       'lifestage_Family Life', 'lifestage_Mature Years',
       'lifestage_Younger Years',
       'lifestage_Younger Years Fiscal Fledlings',
       'maritalStatus_Divorced', 'maritalStatus_In registered partnership',
       'maritalStatus_Married',
       'maritalStatus_Single (never in legal unions)',
       'maritalStatus_Widowed', 'education_associate',
       'education_bachelor', 'education_doctoral', 'education_elementary',
       'education_first-professional degree', 'education_mast

In [23]:
print(lm.predict(validX))
print(hr.predict(validX))
print(lr.predict(validX))
print(rr.predict(validX))
print(nn.predict(validX))

[ 1498.17947112  1571.56343927  1405.36056083 ...,  1530.99986249
  2649.38489163  1272.46270401]
[ 1636.35156692  1641.43839124  2158.06261817 ...,  1548.61249935
  2815.27580421  1812.43899235]
[ 1501.88361017  1573.74927951  1414.06079337 ...,  1530.69220105
  2638.12603445  1298.41156453]
[ 1498.21955243  1571.61339239  1405.54086311 ...,  1531.02722852
  2649.32260394  1272.45271344]
[ 1923.64   1987.282  1433.43  ...,  1987.294  2671.85   1944.198]


We could now compare these predictions with the actual targets and calculate the residual sum of squares (RSS) manually. Sklearn, however, simply gives us a related .score() function, which gives us the R2 score that actually derived from the RSS. It equals to 1 - residual sum of square / total sum of squares.

In [24]:
print(lm.score(validX, validY))
print(hr.score(validX, validY))
print(lr.score(validX, validY))
print(rr.score(validX, validY))
print(nn.score(validX, validY))

0.959127691847
0.804845996657
0.95875722738
0.959129791491
0.749849547177


In fact, the Ridge regression model sealed the deal and has the honour of being transformed into a pmml format. But first, it would be nice to plot the actual targets (x-axis) versus the predicted values by this model (y-axis). We do so with the following code.

In [1]:
plt.scatter(validY, rr.predict(validX), color="darkred", marker='.')
plt.title("Predictions vs. Targets")
plt.xlabel('Targets'); plt.ylabel('Predictions'); plt.grid(True)
plt.show()

NameError: name 'plt' is not defined

The plot reveals that none of the high incomes are predicted to be low and vice versa, which is a good thing. The monthly income also appears to be clustered into people earning around 1500, 2500 and 4000€. That is something our regression models did pick up as well, as the predictions on the y-axis are also clustered around these values.

## Export to PMML

Finally, we need to generate the PMML-file based on the Sklearn object. This is done with the sklearn2pmml package. Note that Lily only accepts .xml files.

In [26]:
from sklearn2pmml import sklearn2pmml
sklearn2pmml(rr, "MetricModelDemo.xml")

# Import PMML into Lily
At this point, you should have an xml-file that is ready for import into Lily. Again, this part is well described in the Lily documentation, but for the sake of completeness, I will quickly go over the steps. First of all, you need to add a new model metric by clicking the '+' sign in the Metrics tab. Set all properties, and especially remember to set the data type of your newly predicted variable.

Next, create a model by clicking the '+' sign in the Model tab and switch the configuration type to PMML. In the advanced options, drag your generated xml-file into the box. A list of parameters should appear where you can make sure if Lily recognizes them correctly. Finally, set the output variable to the metric you just created and hit ‘Save’.

You now have created a model metric, created in Python with Sklearn, and imported in Lily via PMML.